In [1]:
import json, os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import zlib
from PIL import Image
import base64
import io

In [2]:
def base64_2_mask(s):
    z = zlib.decompress(base64.b64decode(s))
    n = np.frombuffer(z, np.uint8)
    mask = cv2.imdecode(n, cv2.IMREAD_UNCHANGED)[:, :, 3].astype(bool)
    return mask

In [3]:
def insert_matrix(arr, x_begin, y_begin, new_shape):
    answer = np.copy(arr)
    if y_begin > 0:
        answer = np.vstack((np.zeros((y_begin-1,answer.shape[1])), answer))
    if x_begin > 0:
        answer = np.hstack((np.zeros((answer.shape[0],x_begin-1)), answer))
    if answer.shape[0] < new_shape[0]:
        answer = np.vstack((answer, np.zeros((new_shape[0]-answer.shape[0],answer.shape[1]))))
    if answer.shape[1] < new_shape[1]:
        answer = np.hstack((answer, np.zeros((answer.shape[0], new_shape[1]-answer.shape[1]))))
    return answer

In [4]:
def generate_colour(n):
    if n<3:
        ans = np.zeros(3)
        ans[n] = 255
        return ans
    if n<6:
        ans = 255*np.ones(3)
        ans[n-3] = 0
        return ans
    if n==6:
        return 255*np.ones(3)
    if(n<14):
        return generate_colour(n-7)//2
    np.random.seed(n)
    ans = np.random.randint(0, high=255, size=3)
    return ans

In [5]:
def make_masks(data_dir, res_folder, ann_folder='ann', img_folder='img'):
    ann_files = os.listdir(os.path.join(data_dir, ann_folder))
    for fname in ann_files:
        print("\n", fname)
        with open(os.path.join(data_dir, ann_folder, fname), 'r') as json_file:
            img_name = fname.split('.')[0]
            image = cv2.imread(os.path.join(data_dir, img_folder, img_name + '.png'))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            result = np.zeros(image.shape)
            
            data = json.load(json_file)
            if data['objects']:
                for n, label in enumerate(data['objects']):
                    print(label['id'])
                    mask = np.array(base64_2_mask(label['bitmap']['data']),dtype=int)
                    x_min = label['bitmap']['origin'][0]
                    y_min = label['bitmap']['origin'][1]
                    mask = insert_matrix(mask, x_min, y_min, image.shape)
                    # remove_intersections:
                    result[:,:,0] -= mask*result[:,:,0]
                    result[:,:,1] -= mask*result[:,:,1]
                    result[:,:,2] -= mask*result[:,:,2]
                    colour = generate_colour(n)
                    result[:,:,0] += mask*colour[0]
                    result[:,:,1] += mask*colour[1]
                    result[:,:,2] += mask*colour[2]
                    
                file_name = os.path.join(res_folder, img_name + '_mask' + '.png')
                cv2.imwrite(file_name, result)

In [6]:
make_masks('quality_segmentation/appropriate', 'aaa')


 IMG_6394.png.json
901744584

 IMG_1758.png.json
901868787
901868813
901868895
901868929
901868958
901868989
901869012
901869047
901869089
901869140
901869151
901869288
901869342
901869360
901869371
901869400
901869460
901869481
901869501
901869533
901869576
901869611
901869639
901869698
901869802
901869865
901869896
901869950
901870008
901870062
901870098
901870142

 IMG_6496.png.json
901780234

 IMG_1752.png.json
901700393
901700801
901700830
901701072
901701372
901702037
901702445
901702603

 IMG_1753.png.json
901707915
901708033
901708173
901708265
901708423
901708813
901709081
901709177

 IMG_1751.png.json
901693065
901693799
901696089
901697557

 IMG_6410.png.json
901750539
901750739
901750873
901751034
901751281
901754355
901754518
901755404

 IMG_1759.png.json
901726392
901727118
901727425
901727578

 IMG_1756.png.json
901714369
901714608
901714659
901714883
901714954
901715266
901715389

 IMG_6506.png.json
901714872
901718914
901719899
901720226
901720431
901720697
901721431


KeyboardInterrupt: 